< Part2 - 비지도학습 >
- K-means
- 연관분석

(일단 2유형은 랜포가 국룰이라서 연습한 것에 의의를 둔다)

## K-means
- K개의 군집을 이용해 거리를 구해서 군집화 하는 방법
- iris data set을 활용

### 1.패키지 import

In [61]:
import pandas as pd
import numpy as np
import sklearn

# k-means
from sklearn.cluster import KMeans

### 2.data EDA

In [62]:
# iris data EDA 생략
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/dataset/main/iris.csv')

### 3.데이터 전처리

In [63]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['species'] = le.fit_transform(df['species'])
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


### 4.분석 수행

In [64]:
X = df.drop('species',axis = 1)

In [65]:
# k-means 수행
cluster1 = KMeans(n_clusters = 3, n_init = 10, max_iter = 500, random_state = 42)

# fit
cluster1.fit(X)

KMeans(max_iter=500, n_clusters=3, n_init=10, random_state=42)

In [66]:
# 각 군집의 중심점과 예측 군집 결과값 확인
cluster_center = cluster1.cluster_centers_
cluster_pred = cluster1.predict(X)
print(cluster_center)
print(cluster_pred)

[[5.9016129  2.7483871  4.39354839 1.43387097]
 [5.006      3.428      1.462      0.246     ]
 [6.85       3.07368421 5.74210526 2.07105263]]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 2 2 2 2 0 2 2 2 2
 2 2 0 0 2 2 2 2 0 2 0 2 0 2 2 0 0 2 2 2 2 2 0 2 2 2 2 0 2 2 2 0 2 2 2 0 2
 2 0]


In [67]:
# 기존 데이터에 예측된 군집 결과 붙여 확인
df['cluster1'] = cluster_pred
df

,sepal_length,sepal_width,petal_length,petal_width,species,cluster1
0,5.1,3.5,1.4,0.2,0,1
1,4.9,3.0,1.4,0.2,0,1
2,4.7,3.2,1.3,0.2,0,1
3,4.6,3.1,1.5,0.2,0,1
4,5.0,3.6,1.4,0.2,0,1
...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2,2
146,6.3,2.5,5.0,1.9,2,0
147,6.5,3.0,5.2,2.0,2,2
148,6.2,3.4,5.4,2.3,2,2


### 5.성능평가

In [68]:
# K에 대해 inertia 비교
scope = range(1,10)
inertias = []

for k in scope:
  model = KMeans(n_clusters = k, n_init = 10, max_iter = 500, random_state = 42)
  model.fit(X)
  inertias.append(model.inertia_)
  print(k, inertias[k-1])

# 실행 결과 K가 3부터 급격하게 감소함
# inertia : 각 군집내 오차 제곱 합 -> 군집 내 분산 정의
# elbow 적용 -> inertia가 빠르게 줄어드는 시점의 K를 최적의 값으로 설정

1 681.3705999999996
2 152.34795176035797
3 78.851441426146
4 57.22847321428572
5 46.46117267267268
6 39.03998724608725
7 34.305815295815314
8 30.132440554614483
9 28.29063524195104


## 연관분석
- 항목간 관련성을 파악해 연관 규칙 도출
- 장바구니 분석
- 지지도, 신뢰도, 향상도(lift)

In [69]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

### 1.패키지 import
- 연관분석의 패키지는 mlxtend의 apriori, association_rules를 사용한다

In [70]:
import pandas as pd
import numpy as np

from mlxtend.frequent_patterns import apriori, association_rules

### 2.data EDA

In [71]:
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/dataset/main/retail_dataset.csv')

In [72]:
df.head()

,0,1,2,3,4,5,6
0,Bread,Wine,Eggs,Meat,Cheese,Pencil,Diaper
1,Bread,Cheese,Meat,Diaper,Wine,Milk,Pencil
2,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
3,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
4,Meat,Pencil,Wine,NaN,NaN,NaN,NaN


In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 315 entries, 0 to 314
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       315 non-null    object
 1   1       285 non-null    object
 2   2       245 non-null    object
 3   3       187 non-null    object
 4   4       133 non-null    object
 5   5       71 non-null     object
 6   6       41 non-null     object
dtypes: object(7)
memory usage: 17.4+ KB


### 3.데이터 전처리

In [74]:
# 각 항목에 대해 전체 인코딩 수행
# 장바구니 데이터
items = set()
for col in df:
  items.update(df[col].unique())

In [77]:
# 각 항목에 매칭 - 1, 아니면 0 -> one-hot encoding
itemset = set(items)
encoding = []
for index, row in df.iterrows():
  rowset = set(row)
  labels = {}
  dismatching = list(itemset - rowset)
  matching = list(itemset.intersection(rowset))
  for i in dismatching:
    labels[i] = 0
  for i in matching:
    labels[i] = 1
  encoding.append(labels)
encoding[0]
result = pd.DataFrame(encoding)
result

,NaN,Milk,Bagel,Cheese,Eggs,Wine,Meat,Bread,Diaper,Pencil
0,0,0,0,1,1,1,1,1,1,1
1,0,1,0,1,0,1,1,1,1,1
2,1,1,0,1,1,1,1,0,0,0
3,1,1,0,1,1,1,1,0,0,0
4,1,0,0,0,0,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...
310,1,0,0,1,1,0,0,1,0,0
311,1,1,0,0,0,0,1,0,0,1
312,0,0,0,1,1,1,1,1,1,1
313,1,0,0,1,0,0,1,0,0,0


### 4.분석

In [80]:
# 결측치 제거
result = result.drop(result.columns[0], axis = 1)

In [85]:
# apriori()
freq_items = apriori(result, min_support = 0.2, use_colnames=True)

In [87]:
# assiociation_rules()로 규칙 도출, threshold = 0.6
rules = association_rules(freq_items, metric="confidence", min_threshold = 0.6)
rules.head()
# 신뢰도가 높으면 연관성이 높다!

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(Bagel),(Bread),0.425397,0.504762,0.279365,0.656716,1.301042,1.0,0.064641,1.442650,0.402687,0.429268,0.306831,0.605088
1,(Eggs),(Cheese),0.438095,0.501587,0.298413,0.681159,1.358008,1.0,0.078670,1.563203,0.469167,0.465347,0.360288,0.638048
2,(Wine),(Cheese),0.438095,0.501587,0.269841,0.615942,1.227986,1.0,0.050098,1.297754,0.330409,0.402844,0.229438,0.576958
3,(Meat),(Cheese),0.476190,0.501587,0.323810,0.680000,1.355696,1.0,0.084958,1.557540,0.500891,0.495146,0.357962,0.662785
4,(Cheese),(Meat),0.501587,0.476190,0.323810,0.645570,1.355696,1.0,0.084958,1.477891,0.526414,0.495146,0.323360,0.662785
